In [1]:
import os
import re
import requests
from tqdm import tqdm

CLOUDFLARE_WORKER_URL = "https://cdn.zhuo.li"
DOWNLOAD_PATH = "/media/HDD1"

def download(target_url):
    response = requests.get(f"{CLOUDFLARE_WORKER_URL}?url={target_url}", stream=True)

    if response.status_code == 200:
        file_size = int(response.headers.get("Content-Length", 0))
        # Get the filename from the Content-Disposition header or default to a generic name
        cd = response.headers.get('Content-Disposition')
        if cd:
            fname = re.findall("filename[^;=\n]*=((['\"]).*?\2|[^;\n]*)", cd)
            if len(fname) > 0 and len(fname[0]) > 0:
                fname = fname[0][0].replace('"', '').replace("'", "")
            else:
                fname = os.path.basename(target_url)
        else:
            fname = os.path.basename(target_url)

        # Append the download path to the filename
        fname = os.path.join(DOWNLOAD_PATH, fname)

        progress = tqdm(response.iter_content(chunk_size=1024), f'Downloading {fname}', total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
        with open(fname, 'wb') as f:
            for data in progress.iterable:
                f.write(data)
                progress.update(len(data))
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")

In [ ]:
download("https://download.ifi.uzh.ch/rpg/RVT/datasets/preprocessed/gen4.tar")